In [32]:
# import libraries
from tensorflow import keras
import numpy as np
import os
import time
import datetime
import h5py
import random
from scipy import spatial
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# define the data location and load data
cwd = os.getcwd()
pardir = os.path.dirname(cwd)
output_folder = os.path.join(pardir, "results", "video_pred_quan_eval_results")
data_folder = os.path.join(pardir,"data")
data_path = os.path.join(data_folder,'video_prediction_dataset.hdf5')

print("data_folder:", data_folder)
print("data_path:", data_path)

data_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/data
data_path: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/data/dataset_all/video_prediction_dataset.hdf5


In [3]:
# load testing data
times_test = np.load(os.path.join(data_folder,"times_curr_test.npy"),allow_pickle=True)
print("times_test.shape:", times_test.shape)

with h5py.File(data_path,'r') as f:

    # read in the data
    images_pred_test = f['test']['images_pred'][:,::2,:,:,:]

print("images_pred_test.shape:",images_pred_test.shape)

times_test.shape: (4467,)
images_pred_test.shape: (4467, 8, 64, 64, 3)


In [4]:
# mask out img background
def mask_background(img): # put all background pixels to 0s
    mask_img = img.copy()
    for i in range(64):
        for j in range(64):
            if (i-30)**2+(j-30)**2>=31**2:
                mask_img[:,:,i,j,:]=0
    return mask_img

In [5]:
# normalize image
# mask out the background
mask_images_pred_test = mask_background(images_pred_test)
mask_images_pred_test = mask_images_pred_test.astype('float32')/255
print(mask_images_pred_test.shape)

(4467, 8, 64, 64, 3)


In [6]:
# calculate VGG cosine similarity
vgg16 = keras.applications.VGG16(weights='imagenet',include_top=True,pooling='max',input_shape=(224,224,3))
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
basemodel=keras.Model(inputs=vgg16.input,outputs=keras.layers.Flatten()(vgg16.get_layer('block4_pool').output))

In [8]:
def get_feature_vector3(img):
    img1 = img.reshape(-1,224,224,3)
    feature_vector = np.zeros((img1.shape[0],basemodel.output.shape[-1]))
    batch_size = 100
    num_samples = img1.shape[0]
    indices = np.arange(num_samples)
    for i in range(int(num_samples / batch_size) + 1):
        start_idx = (i * batch_size) % num_samples
        idxs = indices[start_idx:start_idx + batch_size]
        feature_vector[idxs] = basemodel.predict(img1[idxs])
    feature_vector=feature_vector.reshape((img.shape[0],img.shape[1],basemodel.output.shape[-1]))
    return feature_vector

In [9]:
mask_images_pred_test_resized = np.zeros((images_pred_test.shape[0],images_pred_test.shape[1],224,224,3))
for i in range(images_pred_test.shape[0]):
    for j in range(images_pred_test.shape[1]):
        mask_images_pred_test_resized[i,j] = cv2.resize(mask_images_pred_test[i,j],(224,224))
feature_vector_gt_test = (get_feature_vector3(mask_images_pred_test_resized*255)).astype('float32')

In [10]:
def calculate_cosine_similarity(vector1, vector2):
    cosine_similarity = 1-spatial.distance.cosine(vector1,vector2)
    return cosine_similarity

In [11]:
feature_vector_gt_test.dtype

dtype('float32')

### Evaluation of model performance quantitatively

### VideoGPT

In [12]:
# video prediction models for predicted images
def loadData(samp_num, vp_model, vp_model_folder):

    print("vp_model_folder:", vp_model_folder)

    # load predicted images
    predicted_images = np.load(os.path.join(vp_model_folder,"sample_" + str(samp_num) + ".npy"))[:,8:, :, :, :]
    predicted_images = mask_background(predicted_images.astype('float32'))
    
    predicted_images_resized = np.zeros((predicted_images.shape[0],predicted_images.shape[1],224,224,3))
    for i in range(predicted_images.shape[0]):
        for j in range(predicted_images.shape[1]):
            predicted_images_resized[i,j] = cv2.resize(predicted_images[i,j],(224,224))
    feature_vector_pred_test = (get_feature_vector3(predicted_images_resized*255)).astype('float32')
    print("predicted_images.shape:",predicted_images.shape)
    print("feature_vector_pred_test.shape:",feature_vector_pred_test.shape)
    
    return predicted_images, feature_vector_pred_test

In [24]:
from tqdm.notebook import tqdm

def metrics_batch(predicted_images, images_pred_test, feature_vector_gt, feature_vector_pred):
    mse_per_frame = np.mean(
        np.min(
            np.sum(
                (predicted_images-images_pred_test[None])**2,
                axis=(3,4,5)
                ),
            axis=0),
        axis=0
    )
    mae_per_frame = np.mean(
        np.min(
            np.sum(
                np.abs(predicted_images-images_pred_test[None]),
                axis=(3,4,5)
            ),
        axis=0),
    axis=0)
    
    
    cosine_similarity_per_frame = np.zeros((predicted_images.shape[0],images_pred_test.shape[0],images_pred_test.shape[1]))
    
    for i in range(images_pred_test.shape[0]):
        for j in range(images_pred_test.shape[1]):
            for k in range(predicted_images.shape[0]):
                cosine_similarity_per_frame[k,i,j] = calculate_cosine_similarity(feature_vector_gt[i,j],feature_vector_pred[k,i,j])
    

    
    cosine_similarity_on_time_step = np.mean(np.max(cosine_similarity_per_frame, axis=0),axis=0)
    
    # take mean across all time stamps
    cosine_similarity_per_frame_mean = np.mean(cosine_similarity_per_frame, axis=2)
    
    # find the best match across all 10 samplings and take the mean of every sample
    cosine_similarity_per_frame_mean = np.mean(np.max(cosine_similarity_per_frame_mean,axis=0),axis=0)
    
    # find the index of the best/worst match across 10 samplings for every sample
    best_match_img_index = np.argmax(cosine_similarity_per_frame_mean,axis=0)
    worst_match_img_index = np.argmin(cosine_similarity_per_frame_mean,axis=0)
    
    np.save(os.path.join(output_folder,'VideoGPT_4x4x4_mse_min'),mse_per_frame)
    np.save(os.path.join(output_folder,'VideoGPT_4x4x4_mae_min'),mae_per_frame)
    np.save(os.path.join(output_folder,'VideoGPT_4x4x4_cosine_similarity_max_block4pool'),cosine_similarity_on_time_step)
    np.save(os.path.join(output_folder,'best_match_img_index_based_on_cosine_similarity_among_10sampling_VideoGPT_4x4x4_block4_pool_features.npy'), best_match_img_index)
    np.save(os.path.join(output_folder,'worst_match_img_index_based_on_cosine_similarity_among_10sampling_VideoGPT_4x4x4_block4_pool_features.npy'), worst_match_img_index)
    
    for j in range(images_pred_test.shape[1]):
        print('-'*50)
        print('evaluation for {0} min'.format(j*2+1))
        print('mse (frame-wise): ', mse_per_frame[j])
        print('mae (frame-wise): ', mae_per_frame[j])
        print('cosine_similarity: ', cosine_similarity_on_time_step[j])
        
    print("*"*50)
    print('overall performance:')
    print('mse (frame-wise): ', np.mean(mse_per_frame))
    print('mae (frame-wise): ', np.mean(mae_per_frame))
    print('cosine_similarity: ', np.mean(cosine_similarity_on_time_step))
    
    return cosine_similarity_per_frame, best_match_img_index, worst_match_img_index

In [16]:
#loop over all samples
samp_array  = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
print(images_pred_test.shape)

predicted_images_batch = []
feature_vector_pred_test_batch = []
for samp_num in samp_array:
    vp_model = "VideoGPT_2min_4x4x4_sample_" + str(samp_num)
    vp_model_folder = '/scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/VideoGPT_full_2min'
    print('Sample #: ', samp_num)
    predicted_images, feature_vector_pred_test = loadData(samp_num, vp_model, vp_model_folder)
    predicted_images_batch.append(predicted_images.astype('float32'))
    feature_vector_pred_test_batch.append(feature_vector_pred_test)

(4467, 8, 64, 64, 3)
Sample #:  0
vp_model_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/VideoGPT_full_2min
predicted_images.shape: (4467, 8, 64, 64, 3)
feature_vector_pred_test.shape: (4467, 8, 100352)
Sample #:  1
vp_model_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/VideoGPT_full_2min
predicted_images.shape: (4467, 8, 64, 64, 3)
feature_vector_pred_test.shape: (4467, 8, 100352)
Sample #:  2
vp_model_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/VideoGPT_full_2min
predicted_images.shape: (4467, 8, 64, 64, 3)
feature_vector_pred_test.shape: (4467, 8, 100352)
Sample #:  3
vp_model_folder: /scratch/groups/abrandt/solar_forecasting/GAN_project_new/models/VideoGPT/inference/VideoGPT_full_2min
predicted_images.shape: (4467, 8, 64, 64, 3)
feature_vector_pred_test.shape: (4467, 8, 100352)
Sample #:  4
vp_model_folder: /scratch/groups/abrandt/solar_forecas

In [25]:
cosine_similarity, best_match_img_index, worst_match_img_index = metrics_batch(np.stack(predicted_images_batch), mask_images_pred_test, feature_vector_gt_test, np.stack(feature_vector_pred_test_batch))

--------------------------------------------------
evaluation for 1 min
mse (frame-wise):  27.720184
mae (frame-wise):  302.92438
ssim:  0.8823216264631143
cosine_similarity:  0.7624971956141395
--------------------------------------------------
evaluation for 3 min
mse (frame-wise):  38.532486
mae (frame-wise):  362.92456
ssim:  0.8556056885090755
cosine_similarity:  0.7438174404632546
--------------------------------------------------
evaluation for 5 min
mse (frame-wise):  45.655457
mae (frame-wise):  401.26743
ssim:  0.8402210531885606
cosine_similarity:  0.7313974852633471
--------------------------------------------------
evaluation for 7 min
mse (frame-wise):  50.729412
mae (frame-wise):  429.15485
ssim:  0.8306120032148608
cosine_similarity:  0.7222640416166888
--------------------------------------------------
evaluation for 9 min
mse (frame-wise):  54.91339
mae (frame-wise):  449.88254
ssim:  0.8234377169121144
cosine_similarity:  0.7146154165214571
--------------------------

In [26]:
predicted_images_batch_stacked = np.stack(predicted_images_batch)
feature_vector_pred_test_batch_stacked = np.stack(feature_vector_pred_test_batch)

In [ ]:
np.save(os.path.join(output_folder,'predicted_images_10sampling_stacked_mask_background_VideoGPT_4x4x4.npy'),predicted_images_batch_stacked)